In [ ]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

In [ ]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim")
data_module.setup(stage="fit")
dl = data_module.train_dataloader()